In [1]:
import sys
sys.path.append("../src")
import os

import pandas as pd
import numpy as np
from etf_transformations import *
from pathlib import Path

from sklearn.model_selection import train_test_split, StratifiedKFold, RandomizedSearchCV
from catboost import CatBoostClassifier
from sklearn.metrics import classification_report, roc_auc_score, confusion_matrix
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.model_selection import RandomizedSearchCV
from scipy.stats import uniform, randint
from sklearn.preprocessing import FunctionTransformer

In [2]:
df = pd.read_csv("../data/preprocessed/final_etf_data/weekends_aggregated_embedding/XLV_v4.csv")
df = sign_next_day(df)
df = drop_sign_and_return(df)


In [3]:
df.head(10)

,Date,Price,avg_positive_XLV,avg_neutral_XLV,avg_negative_XLV,n_XLV,sent_index_XLV,emb_0,emb_1,emb_2,...,emb_376,emb_377,emb_378,emb_379,emb_380,emb_381,emb_382,emb_383,is_trading_day,Sign_next_day
0,2018-03-20,74.048599,0.774735,0.086462,0.138803,1.0,0.635933,0.014913,-0.088325,-0.062096,...,-0.010546,-0.007227,0.001823,-0.034662,0.075100,-0.084690,0.035918,-0.006692,1.0,-1.0
1,2018-03-21,73.731720,0.065696,0.924038,0.010265,1.0,0.055431,-0.054664,-0.050197,-0.069135,...,-0.028869,-0.029211,-0.017493,-0.048448,0.011812,-0.081526,0.020009,-0.067594,1.0,-1.0
2,2018-03-22,71.610535,0.111366,0.724991,0.163644,2.0,-0.052278,-0.090927,0.013682,-0.019073,...,-0.083093,-0.033170,-0.013238,-0.067610,0.086293,-0.099265,-0.044316,0.040492,1.0,-1.0
3,2018-03-23,70.123039,0.043882,0.149664,0.806454,1.0,-0.762573,-0.080639,0.046290,-0.051142,...,-0.085956,-0.047095,-0.031485,-0.091243,0.029235,-0.188138,-0.037388,0.055705,1.0,1.0
4,2018-03-26,71.575317,0.219635,0.764985,0.015380,1.0,0.204255,-0.014982,-0.037512,-0.001159,...,-0.053044,0.015748,0.026849,0.009882,0.084287,-0.066474,-0.033048,0.002425,1.0,-1.0
5,2018-03-27,70.730339,0.117630,0.841624,0.040745,1.0,0.076885,-0.058472,0.021489,0.016755,...,-0.047308,-0.027205,-0.039886,-0.031570,0.036515,-0.121883,-0.012859,0.033212,1.0,1.0
6,2018-03-28,71.108849,0.535672,0.350854,0.113474,2.0,0.422198,-0.017524,0.017952,-0.004189,...,0.001684,-0.022777,-0.056764,0.051483,0.013841,-0.095180,0.022969,0.051427,1.0,1.0
7,2018-03-29,71.645737,0.017688,0.201077,0.781234,1.0,-0.763546,-0.079666,0.025857,-0.077171,...,0.029654,-0.055186,-0.067221,-0.041230,0.001387,-0.057550,-0.028935,0.089678,1.0,-1.0
8,2018-04-02,70.017418,0.040953,0.851402,0.107645,3.0,-0.066691,0.043960,0.026959,-0.010339,...,0.024045,0.053283,-0.021837,0.022628,0.056247,-0.064364,0.071800,-0.031847,1.0,1.0
9,2018-04-03,71.082436,0.047970,0.527828,0.424201,2.0,-0.376231,-0.045560,-0.019583,-0.022873,...,-0.043971,-0.045721,-0.033195,-0.031067,-0.000990,-0.055234,0.026710,0.032353,1.0,1.0


In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 587 entries, 0 to 586
Columns: 393 entries, Date to Sign_next_day
dtypes: float64(392), object(1)
memory usage: 1.8+ MB


In [5]:
df.describe()

,Price,avg_positive_XLV,avg_neutral_XLV,avg_negative_XLV,n_XLV,sent_index_XLV,emb_0,emb_1,emb_2,emb_3,...,emb_376,emb_377,emb_378,emb_379,emb_380,emb_381,emb_382,emb_383,is_trading_day,Sign_next_day
count,587.000000,452.000000,452.000000,452.000000,452.000000,452.000000,452.000000,452.000000,452.000000,452.000000,...,452.000000,452.000000,452.000000,452.000000,452.000000,452.000000,452.000000,452.000000,452.0,586.000000
mean,82.501202,0.223649,0.419172,0.357179,9.112832,-0.133530,-0.007025,0.004606,-0.001310,-0.006124,...,-0.015621,-0.012418,-0.024859,-0.024690,0.014432,-0.098196,0.031132,0.026245,1.0,0.122867
std,6.377453,0.209477,0.257264,0.273857,14.664258,0.414212,0.037740,0.034668,0.032654,0.034582,...,0.035596,0.028692,0.033507,0.038782,0.034935,0.039025,0.042920,0.027829,0.0,0.989823
min,68.337990,0.008496,0.013868,0.007590,1.000000,-0.962044,-0.161993,-0.096628,-0.123921,-0.129486,...,-0.182579,-0.109688,-0.138805,-0.133719,-0.105537,-0.231326,-0.102824,-0.079547,1.0,-1.000000
25%,78.884422,0.066452,0.227062,0.055274,1.000000,-0.410068,-0.027410,-0.015497,-0.018326,-0.027828,...,-0.033009,-0.026950,-0.046329,-0.045759,-0.002609,-0.120299,0.000807,0.011355,1.0,-1.000000
50%,81.641983,0.158298,0.354476,0.376217,2.000000,-0.157523,-0.000007,0.004616,0.005610,-0.002666,...,-0.017739,-0.014527,-0.018445,-0.020674,0.011604,-0.102161,0.025550,0.028691,1.0,1.000000
75%,85.996525,0.308657,0.594141,0.547057,10.000000,0.099106,0.016890,0.022449,0.018479,0.016226,...,0.000163,0.001666,0.000442,-0.005802,0.031941,-0.076358,0.056383,0.041541,1.0,1.000000
max,97.386276,0.941824,0.949438,0.974088,85.000000,0.920362,0.123103,0.138262,0.118386,0.115019,...,0.114770,0.099031,0.079649,0.125455,0.135212,0.042213,0.180569,0.118798,1.0,1.000000


In [6]:
df.isna().sum()

Date                  0
Price                 0
avg_positive_XLV    135
avg_neutral_XLV     135
avg_negative_XLV    135
                   ... 
emb_381             135
emb_382             135
emb_383             135
is_trading_day      135
Sign_next_day         1
Length: 393, dtype: int64

In [7]:
df.nunique()

Date                587
Price               567
avg_positive_XLV    452
avg_neutral_XLV     452
avg_negative_XLV    452
                   ... 
emb_381             452
emb_382             452
emb_383             452
is_trading_day        1
Sign_next_day         3
Length: 393, dtype: int64

### defining target - I want to assign:

class 1 to sign_next_day ==1

and class 0 to sign_next_day ∈ {0, -1}

In [8]:
df = df.dropna(subset=["Sign_next_day"])
df["Sign_next_day"] = (df["Sign_next_day"] == 1).astype(int)
df.head()

,Date,Price,avg_positive_XLV,avg_neutral_XLV,avg_negative_XLV,n_XLV,sent_index_XLV,emb_0,emb_1,emb_2,...,emb_376,emb_377,emb_378,emb_379,emb_380,emb_381,emb_382,emb_383,is_trading_day,Sign_next_day
0,2018-03-20,74.048599,0.774735,0.086462,0.138803,1.0,0.635933,0.014913,-0.088325,-0.062096,...,-0.010546,-0.007227,0.001823,-0.034662,0.075100,-0.084690,0.035918,-0.006692,1.0,0
1,2018-03-21,73.731720,0.065696,0.924038,0.010265,1.0,0.055431,-0.054664,-0.050197,-0.069135,...,-0.028869,-0.029211,-0.017493,-0.048448,0.011812,-0.081526,0.020009,-0.067594,1.0,0
2,2018-03-22,71.610535,0.111366,0.724991,0.163644,2.0,-0.052278,-0.090927,0.013682,-0.019073,...,-0.083093,-0.033170,-0.013238,-0.067610,0.086293,-0.099265,-0.044316,0.040492,1.0,0
3,2018-03-23,70.123039,0.043882,0.149664,0.806454,1.0,-0.762573,-0.080639,0.046290,-0.051142,...,-0.085956,-0.047095,-0.031485,-0.091243,0.029235,-0.188138,-0.037388,0.055705,1.0,1
4,2018-03-26,71.575317,0.219635,0.764985,0.015380,1.0,0.204255,-0.014982,-0.037512,-0.001159,...,-0.053044,0.015748,0.026849,0.009882,0.084287,-0.066474,-0.033048,0.002425,1.0,0


In [9]:
df.shape

(586, 393)

In [10]:
df.isna().sum()

Date                  0
Price                 0
avg_positive_XLV    135
avg_neutral_XLV     135
avg_negative_XLV    135
                   ... 
emb_381             135
emb_382             135
emb_383             135
is_trading_day      135
Sign_next_day         0
Length: 393, dtype: int64

In [11]:
df.head()

,Date,Price,avg_positive_XLV,avg_neutral_XLV,avg_negative_XLV,n_XLV,sent_index_XLV,emb_0,emb_1,emb_2,...,emb_376,emb_377,emb_378,emb_379,emb_380,emb_381,emb_382,emb_383,is_trading_day,Sign_next_day
0,2018-03-20,74.048599,0.774735,0.086462,0.138803,1.0,0.635933,0.014913,-0.088325,-0.062096,...,-0.010546,-0.007227,0.001823,-0.034662,0.075100,-0.084690,0.035918,-0.006692,1.0,0
1,2018-03-21,73.731720,0.065696,0.924038,0.010265,1.0,0.055431,-0.054664,-0.050197,-0.069135,...,-0.028869,-0.029211,-0.017493,-0.048448,0.011812,-0.081526,0.020009,-0.067594,1.0,0
2,2018-03-22,71.610535,0.111366,0.724991,0.163644,2.0,-0.052278,-0.090927,0.013682,-0.019073,...,-0.083093,-0.033170,-0.013238,-0.067610,0.086293,-0.099265,-0.044316,0.040492,1.0,0
3,2018-03-23,70.123039,0.043882,0.149664,0.806454,1.0,-0.762573,-0.080639,0.046290,-0.051142,...,-0.085956,-0.047095,-0.031485,-0.091243,0.029235,-0.188138,-0.037388,0.055705,1.0,1
4,2018-03-26,71.575317,0.219635,0.764985,0.015380,1.0,0.204255,-0.014982,-0.037512,-0.001159,...,-0.053044,0.015748,0.026849,0.009882,0.084287,-0.066474,-0.033048,0.002425,1.0,0


In [12]:
df_catboost = df.copy()
df_catboost.head()

,Date,Price,avg_positive_XLV,avg_neutral_XLV,avg_negative_XLV,n_XLV,sent_index_XLV,emb_0,emb_1,emb_2,...,emb_376,emb_377,emb_378,emb_379,emb_380,emb_381,emb_382,emb_383,is_trading_day,Sign_next_day
0,2018-03-20,74.048599,0.774735,0.086462,0.138803,1.0,0.635933,0.014913,-0.088325,-0.062096,...,-0.010546,-0.007227,0.001823,-0.034662,0.075100,-0.084690,0.035918,-0.006692,1.0,0
1,2018-03-21,73.731720,0.065696,0.924038,0.010265,1.0,0.055431,-0.054664,-0.050197,-0.069135,...,-0.028869,-0.029211,-0.017493,-0.048448,0.011812,-0.081526,0.020009,-0.067594,1.0,0
2,2018-03-22,71.610535,0.111366,0.724991,0.163644,2.0,-0.052278,-0.090927,0.013682,-0.019073,...,-0.083093,-0.033170,-0.013238,-0.067610,0.086293,-0.099265,-0.044316,0.040492,1.0,0
3,2018-03-23,70.123039,0.043882,0.149664,0.806454,1.0,-0.762573,-0.080639,0.046290,-0.051142,...,-0.085956,-0.047095,-0.031485,-0.091243,0.029235,-0.188138,-0.037388,0.055705,1.0,1
4,2018-03-26,71.575317,0.219635,0.764985,0.015380,1.0,0.204255,-0.014982,-0.037512,-0.001159,...,-0.053044,0.015748,0.026849,0.009882,0.084287,-0.066474,-0.033048,0.002425,1.0,0


In [13]:
df_catboost["Date"] = pd.to_datetime(df_catboost["Date"]).dt.normalize()
df_catboost = df_catboost.sort_values("Date").reset_index(drop=True)

df_catboost["no_news"] = df_catboost["n_XLV"].isna().astype(int) # a day where there were no sector specific news
df_catboost.head(10)


,Date,Price,avg_positive_XLV,avg_neutral_XLV,avg_negative_XLV,n_XLV,sent_index_XLV,emb_0,emb_1,emb_2,...,emb_377,emb_378,emb_379,emb_380,emb_381,emb_382,emb_383,is_trading_day,Sign_next_day,no_news
0,2018-03-20,74.048599,0.774735,0.086462,0.138803,1.0,0.635933,0.014913,-0.088325,-0.062096,...,-0.007227,0.001823,-0.034662,0.075100,-0.084690,0.035918,-0.006692,1.0,0,0
1,2018-03-21,73.731720,0.065696,0.924038,0.010265,1.0,0.055431,-0.054664,-0.050197,-0.069135,...,-0.029211,-0.017493,-0.048448,0.011812,-0.081526,0.020009,-0.067594,1.0,0,0
2,2018-03-22,71.610535,0.111366,0.724991,0.163644,2.0,-0.052278,-0.090927,0.013682,-0.019073,...,-0.033170,-0.013238,-0.067610,0.086293,-0.099265,-0.044316,0.040492,1.0,0,0
3,2018-03-23,70.123039,0.043882,0.149664,0.806454,1.0,-0.762573,-0.080639,0.046290,-0.051142,...,-0.047095,-0.031485,-0.091243,0.029235,-0.188138,-0.037388,0.055705,1.0,1,0
4,2018-03-26,71.575317,0.219635,0.764985,0.015380,1.0,0.204255,-0.014982,-0.037512,-0.001159,...,0.015748,0.026849,0.009882,0.084287,-0.066474,-0.033048,0.002425,1.0,0,0
5,2018-03-27,70.730339,0.117630,0.841624,0.040745,1.0,0.076885,-0.058472,0.021489,0.016755,...,-0.027205,-0.039886,-0.031570,0.036515,-0.121883,-0.012859,0.033212,1.0,1,0
6,2018-03-28,71.108849,0.535672,0.350854,0.113474,2.0,0.422198,-0.017524,0.017952,-0.004189,...,-0.022777,-0.056764,0.051483,0.013841,-0.095180,0.022969,0.051427,1.0,1,0
7,2018-03-29,71.645737,0.017688,0.201077,0.781234,1.0,-0.763546,-0.079666,0.025857,-0.077171,...,-0.055186,-0.067221,-0.041230,0.001387,-0.057550,-0.028935,0.089678,1.0,0,0
8,2018-04-02,70.017418,0.040953,0.851402,0.107645,3.0,-0.066691,0.043960,0.026959,-0.010339,...,0.053283,-0.021837,0.022628,0.056247,-0.064364,0.071800,-0.031847,1.0,1,0
9,2018-04-03,71.082436,0.047970,0.527828,0.424201,2.0,-0.376231,-0.045560,-0.019583,-0.022873,...,-0.045721,-0.033195,-0.031067,-0.000990,-0.055234,0.026710,0.032353,1.0,1,0


In [14]:
df_catboost = df_catboost.drop(columns=["Date"])
df_catboost.head(1)

,Price,avg_positive_XLV,avg_neutral_XLV,avg_negative_XLV,n_XLV,sent_index_XLV,emb_0,emb_1,emb_2,emb_3,...,emb_377,emb_378,emb_379,emb_380,emb_381,emb_382,emb_383,is_trading_day,Sign_next_day,no_news
0,74.048599,0.774735,0.086462,0.138803,1.0,0.635933,0.014913,-0.088325,-0.062096,-0.014948,...,-0.007227,0.001823,-0.034662,0.0751,-0.08469,0.035918,-0.006692,1.0,0,0


In [15]:
original_features = [
    'Price',
    'avg_positive_XLV',
    'avg_neutral_XLV', 
    'avg_negative_XLV',
    'n_XLV',
    'sent_index_XLV',
    'is_trading_day'
]


embedding_features = [f'emb_{i}' for i in range(384)]

target = 'Sign_next_day'
df_clean = df_catboost.dropna(subset=[target, 'is_trading_day']).copy()


for emb_col in embedding_features:
    df_clean[emb_col] = df_clean[emb_col].fillna(0)


df_clean['avg_positive_XLV'] = df_clean['avg_positive_XLV'].fillna(0)
df_clean['avg_neutral_XLV'] = df_clean['avg_neutral_XLV'].fillna(1)
df_clean['avg_negative_XLV'] = df_clean['avg_negative_XLV'].fillna(0)
df_clean['n_XLV'] = df_clean['n_XLV'].fillna(0)
df_clean['sent_index_XLV'] = df_clean['sent_index_XLV'].fillna(0)

print(f"Clean dataset size: {len(df_clean)} rows")


Clean dataset size: 451 rows


In [16]:
X = df_clean[original_features + embedding_features]
y = df_clean[target]

#y = (y == 1).astype(int)

print(f"Target distribution: {y.value_counts().to_dict()}")

Target distribution: {1: 249, 0: 202}


In [17]:
embedding_pipeline = Pipeline([
    ('scaler', StandardScaler()),
    ('pca', PCA(n_components=0.95, random_state=12))  # 95% variance
])


original_pipeline = Pipeline([
    ('passthrough', FunctionTransformer())
])

preprocessor = ColumnTransformer(
    transformers=[
        ('original', original_pipeline, original_features),
        ('embeddings', embedding_pipeline, embedding_features)
    ],
    remainder='drop'
)

In [18]:
catboost = CatBoostClassifier(
    loss_function="Logloss",
    eval_metric="AUC",
    random_seed=12,
    verbose=0,
    train_dir="cb_tmp",
    allow_writing_files=False
)

full_pipeline = Pipeline([
    ('preprocessor', preprocessor),
    ('classifier', catboost)
])


param_distributions = {
    'preprocessor__embeddings__pca__n_components': [0.90, 0.95, 0.98, 50, 75, 100],
    'classifier__depth': randint(3, 7),
    'classifier__learning_rate': uniform(0.01, 0.19),
    'classifier__iterations': randint(50, 500),
    'classifier__l2_leaf_reg': uniform(1, 14),
    'classifier__border_count': randint(32, 128),
    'classifier__random_strength': uniform(0.5, 5),
    'classifier__min_data_in_leaf': randint(5, 30),
    'classifier__rsm': uniform(0.6, 0.4),
    'classifier__boosting_type': ['Ordered', 'Plain'],
    'classifier__bootstrap_type': ['Bernoulli', 'MVS'],
    'classifier__subsample': uniform(0.6, 0.4),
}

random_search = RandomizedSearchCV(
    estimator=full_pipeline,
    param_distributions=param_distributions,
    n_iter=40,
    cv=5,
    scoring='roc_auc',
    random_state=12,
    n_jobs=-1,
    verbose=2,
    return_train_score=True
)


random_search.fit(X, y)

print(f"Best AUC: {random_search.best_score_:.4f}")
print(f"Best parameters:")
for param, value in random_search.best_params_.items():
    print(f"  {param}: {value}")



Fitting 5 folds for each of 40 candidates, totalling 200 fits
Best AUC: 0.5177
Best parameters:
  classifier__boosting_type: Ordered
  classifier__bootstrap_type: MVS
  classifier__border_count: 52
  classifier__depth: 5
  classifier__iterations: 409
  classifier__l2_leaf_reg: 12.210279279017385
  classifier__learning_rate: 0.18317246952533064
  classifier__min_data_in_leaf: 11
  classifier__random_strength: 2.546660714246617
  classifier__rsm: 0.9145114433093094
  classifier__subsample: 0.705195288070581
  preprocessor__embeddings__pca__n_components: 100


In [19]:
best_preprocessor = random_search.best_estimator_.named_steps['preprocessor']
pca = best_preprocessor.named_transformers_['embeddings'].named_steps['pca']

print(f"Number of prncipals components kept: {pca.n_components_}")
print(f"Explained variance ratio: {pca.explained_variance_ratio_.sum():.4f}")
print(f"Original embedding dimensions (384) were reduced to {pca.n_components_}")


Number of prncipals components kept: 100
Explained variance ratio: 0.8924
Original embedding dimensions (384) were reduced to 100
